# 제조보고서 원단 '타입/반품', '사용' 내역찾기

# 프로그램 개요

- 아래 프로그램은 제조보고서 2022년 04월 'FQMADU' 아이템의 기말재고가 -0.5 야드가 나와서 원인규명을 위해 제작했다.
- 제조보고서 아이템(완제품)중 바지나 치마같은 제품은 물량이 많아 문제가 생겼을 때 원단 추적이 까다롭다.
- 타입/반품과 사용은 + - 관계
- 타입 총량과 사용 총량을 sum 하여 튀어나오는 order를 잡아낸다.
- 튀어나온 오더로 재검색하고 입출고 내역을 확인하면 문제가 생긴 시점을 알 수 있다.
    - 원래 쿼리에서 group by로 sum한 쿼리를 개별건으로 해체해놔서 검색하면 모든 개별 order 내역이 나온다.
- 덤으로 회수안된 샘플오더가 로직상 같이 걸려나온다.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import binascii   # 한글 변환에 필요한 라이브러리
import sys
import pyautogui
from datetime import datetime
from typing import List

### DB 접속

In [2]:
sys.path.append('/settings')
import config

def connect_db(sid):
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
    
    conn = create_engine(
        "oracle+cx_oracle://{user}:{password}@{host}:{port}/{sid}?encoding=UTF-8&nencoding=UTF-8".format(
            user=config.COMPANY_DB_CONFIG['user'],
            password=config.COMPANY_DB_CONFIG['password'],
            host=config.COMPANY_DB_CONFIG['host'],
            port=config.COMPANY_DB_CONFIG['port'],
            sid=config.COMPANY_DB_CONFIG['sid']
        )
    )
    
    return conn

engine = connect_db('IVY')

### 완제품품번

In [3]:
items: str = pyautogui.prompt(title='아이템',
                         text='제조보고서의 확인할 아이템(완제품 품번)을 입력하세요.\n여러개 입력시 , 로 구분하세요.\n예 : FQMADU, FJFBDU, FHMBAA',
                         default='FQMADU')
items = items.replace(' ', '').upper()
items

'SNFCAG,SNMCAG,SZMATS'

In [4]:
items: List[str] = items.split(',')
items

['SNFCAG', 'SNMCAG', 'SZMATS']

#### 오늘 날짜

In [5]:
this_year = datetime.today().strftime('%Y')
this_month = datetime.today().strftime('%m')
last_year = str(int(this_year)-1)

this_year, this_month, last_year

('2022', '07', '2021')

### 시작월

In [6]:
start_month = pyautogui.prompt(title='조회기간 시작 월',
                              text=f'조회기간 시작 월을 입력하세요.\n예 : {last_year+"06"}',
                              default=f'{last_year+"06"}')
start_month

'202106'

### 종료월

In [7]:
end_month = pyautogui.prompt(title='조회기간 종료 월',
                             text=f'조회기간 종료 월을 입력하세요.\n예 : {this_year + this_month}',
                             default=f'{this_year + this_month}')
end_month

'202205'

### 한글변환 함수

In [8]:
# # US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df: pd.DataFrame) -> pd.DataFrame:
    for index, byte_data in enumerate(df):
        if byte_data == None: # null 값이면 패스. 안하면 변환 에러난다.
            continue
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
    return df

### 표준정보 쿼리 함수

In [9]:
def sql_query(oracleSql: str) -> pd.DataFrame:
    df = pd.read_sql_query(oracleSql, engine)

    if 'tkyk_name' in df.columns: # 해당컬럼이 없어도 에러없이 처리
        df_temp = df['tkyk_name'].copy()
        df['tkyk_name'] = us7ascii_to_cp949(df_temp)

    if 'agen_name' in df.columns:
        df_temp = df['agen_name'].copy()
        df['agen_name'] = us7ascii_to_cp949(df_temp)

    if 'sch_name' in df.columns:
        df_temp = df['sch_name'].copy()
        df['sch_name'] = us7ascii_to_cp949(df_temp)
    
    return df

### 조건부서식 함수 (셀 스타일 배경색 추가, 텍스트 굵게 적용)

In [10]:
def draw_color_in_bursojae(value: str, color: str, font_weight: str) -> str:
    if str(value) == '00':
        fill1 = color
        fill2 = font_weight
    else:
        fill1 = "inherit"
        fill2 = "inherit"

    return f"background-color: {fill1};font-weight: {fill2}"

### 해당 ITEM 의 완전히 소진되지 않은 원단찾는 함수

In [11]:
def get_fabric_diff(item: str) -> None:
    
    # 타입/반품 SQL

    # i_chulgo_t 테이블
    #  cdate 기준으로 정산
    # 기간 : 전년 6월 ~ 문제생긴 달
    # qty 가 양수이면 타입, 음수면 반품
    
    sql1 = f'''
SELECT '1'                              gubun,
       master_order                     order_no,
       master_status                    st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       chulgo_item                      item,
       stock_paitem                     paitem,
       chulgo_sojae                     sojae,
       chulgo_millon                    millon,
       chulgo_cdate                     cdate,
       Decode(chulgo_fact, 'VPF22', 'VPF31',
                           chulgo_fact) fact,
       chulgo_qty                       qty
FROM   i_chulgo_t,
       i_stock_t,
       i_suju_master_t,
       i_sch_t
WHERE  chulgo_cdate BETWEEN To_date('{start_month[:4]}'
                                    ||'{start_month[-2:]}'
                                    ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                            Last_day(
                                                        To_date('{end_month[:4]}'
                                                                ||'{end_month[-2:]}'
                                                                ||'01 235959',
                                                        'yyyymmdd hh24:mi:ss'))
       AND chulgo_item IN ('{item}')
       AND chulgo_millon = stock_millon
       AND chulgo_order = master_order
       AND ( master_remake = 'H'
              OR master_agent = 'VZ001' )
       AND sch_code(+) = master_school
UNION ALL
SELECT '2'                              gubun,
       master2_order                    order_no,
       master2_status                   st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       chulgo_item                      item,
       stock_paitem                     paitem,
       chulgo_sojae                     sojae,
       chulgo_millon                    millon,
       chulgo_cdate                     cdate,
       Decode(chulgo_fact, 'VPF22', 'VPF31',
                           chulgo_fact) fact,
       chulgo_qty                       qty
FROM   i_chulgo_t,
       i_stock_t,
       i_suju_master2_t,
       i_sch_t
WHERE  chulgo_cdate BETWEEN To_date('{start_month[:4]}'
                                    ||'{start_month[-2:]}'
                                    ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                            Last_day(
                                                        To_date('{end_month[:4]}'
                                                                ||'{end_month[-2:]}'
                                                                ||'01 235959',
                                                        'yyyymmdd hh24:mi:ss'))
       AND chulgo_item IN ('{item}')
       AND chulgo_millon = stock_millon
       AND chulgo_order = master2_order
       AND master2_remake = 'S'
       AND master2_jaepum <> 'P'
       AND master2_squota IS NOT NULL
       AND sch_code(+) = master2_school
UNION ALL
SELECT '3'                              gubun,
       master_order                     order_no,
       master_status                    st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       chulgo_item                      item,
       stock_paitem                     paitem,
       chulgo_sojae                     sojae,
       chulgo_millon                    millon,
       chulgo_cdate                     cdate,
       Decode(chulgo_fact, 'VPF22', 'VPF31',
                           chulgo_fact) fact,
       chulgo_qty                       qty
FROM   i_chulgo_t,
       i_stock_t,
       i_suju_master_t,
       i_sch_t
WHERE  chulgo_cdate BETWEEN To_date('{start_month[:4]}'
                                    ||'{start_month[-2:]}'
                                    ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                            Last_day(
                                                        To_date('{end_month[:4]}'
                                                                ||'{end_month[-2:]}'
                                                                ||'01 235959',
                                                        'yyyymmdd hh24:mi:ss'))
       AND chulgo_item IN ('{item}')
       AND chulgo_millon = stock_millon
       AND chulgo_order = master_order
       AND master_remake <> 'H'
       AND sch_code(+) = master_school 
'''
    
    # 사용량 SQL

    # 정보가 타입쪽에 비해 모자람.
    # 밀넘버, 소재코드, 봉제업체가 없다.
    
    sql2 = f'''
SELECT '1'                   gubun,
       master_order          order_no,
       master_status         st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       master_item           item,
       prods_paitem          paitem,
       prods_cdate           cdate,
       Nvl(prods_use_qty, 0) qty
FROM   i_suju_fact_t,
       i_prod_sub_t,
       i_suju_master_t,
       i_sch_t
WHERE  prods_cdate BETWEEN To_date('{start_month[:4]}'
                                   ||'{start_month[-2:]}'
                                   ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                           Last_day(
                                                      To_date('{end_month[:4]}'
                                                              ||'{end_month[-2:]}'
                                                              ||'01 235959',
                                                      'yyyymmdd hh24:mi:ss'))
       AND master_item IN ('{item}')
       AND prods_order = fact_order
       AND fact_order = master_order
       AND ( master_remake = 'H'
              OR master_agent = 'VZ001' )
       AND sch_code(+) = master_school
UNION ALL
SELECT '2'                   gubun,
       master2_order         order_no,
       master2_status        st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       master2_item          item,
       prods_paitem          paitem,
       prods_cdate           cdate,
       Nvl(prods_use_qty, 0) qty
FROM   i_suju_fact_t,
       i_prod_sub_t,
       i_suju_master2_t,
       i_sch_t
WHERE  prods_cdate BETWEEN To_date('{start_month[:4]}'
                                   ||'{start_month[-2:]}'
                                   ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                           Last_day(
                                                      To_date('{end_month[:4]}'
                                                              ||'{end_month[-2:]}'
                                                              ||'01 235959',
                                                      'yyyymmdd hh24:mi:ss'))
       AND master2_item IN ('{item}')
       AND prods_order = fact_order
       AND fact_order = master2_order
       AND master2_remake = 'S'
       AND master2_jaepum <> 'P'
       AND master2_squota IS NOT NULL
       AND sch_code(+) = master2_school
UNION ALL
SELECT '3'                   gubun,
       master_order          order_no,
       master_status         st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       master_item           item,
       prods_paitem          paitem,
       prods_cdate           cdate,
       Nvl(prods_use_qty, 0) qty
FROM   i_suju_fact_t,
       i_prod_sub_t,
       i_suju_master_t,
       i_sch_t
WHERE  prods_cdate BETWEEN To_date('{start_month[:4]}'
                                   ||'{start_month[-2:]}'
                                   ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                           Last_day(
                                                      To_date('{end_month[:4]}'
                                                              ||'{end_month[-2:]}'
                                                              ||'01 235959',
                                                      'yyyymmdd hh24:mi:ss'))
       AND master_item IN ('{item}')
       AND prods_order = fact_order
       AND fact_order = master_order
       AND master_remake <> 'H'
       AND master_agent <> 'VZ001'
       AND sch_code(+) = master_school
'''
    
    # 타입/반품 쿼리
    df_taip = sql_query(sql1)
    df_ta = df_taip[['order_no','st','sch_name','item','paitem','sojae','millon','cdate','fact','qty']].copy()

    # 사용량 쿼리
    df_sayoung = sql_query(sql2)
    df_sayoung['millon'] = '' # 프레임 합치기를 위해 빈 column 생성
    df_sayoung['fact'] = ''
    df_sayoung['sojae'] = ''
    df_sa = df_sayoung[['order_no','st','sch_name','item','paitem','sojae','millon','cdate','fact','qty']].copy()
    df_sa['qty'] = df_sa['qty'] * (-1) # sum해서 0으로 만들거라 -1을 곱해준다.
    
    # 합치기
    df_calc = pd.concat([df_ta, df_sa])
    
    # 집계
    df_calc2 = df_calc.groupby(['item', 'order_no'])[['qty']].agg(sum) # 아이템과 오더번호 기준 sum
    
    # 집계결과에서 합계가 0인 오더는 삭제
    df_result = df_calc2[df_calc2['qty'] != 0] # 0인 오더를 걷어낸다. (집계내용이 없으면 에러)
    
    df_result = df_result.reset_index()
    order = df_result['order_no'].to_list()
    
    # 세부내역
    df_temp1 = pd.DataFrame()
    for no in order:
        df_temp1 = pd.concat([df_temp1, df_ta[df_ta['order_no'] == no]])
    df_temp1 = df_temp1.sort_values(['cdate']).reset_index(drop=True)
    
    # 혹시나 사용량도 확인해본다.
    df_temp2 = pd.DataFrame()
    for no in order:
        df_temp2 = pd.concat([df_temp2, df_sa[df_sa['order_no'] == no]])
    
    # 조건부서식 적용 (셀 스타일 배경색 추가, 텍스트 굵게 적용)
    df_temp1 = df_temp1.style.applymap(draw_color_in_bursojae, color='#FFC000', font_weight='bold')
    
    # 시트 기록
    df_temp1.to_excel(writer, sheet_name=item, index=False)

In [12]:
# 엑셀저장
writer = pd.ExcelWriter(f'{start_month}_{end_month}_제조보고서_원단내역.xlsx', engine='xlsxwriter')

for item in items:
    get_fabric_diff(item)

writer.save()